In [1]:
import os
from pathlib import Path

current_directory = Path.cwd()
if current_directory.name == "MT":
    # This means that the notebook is run from the main anomalib directory.
    root_directory = current_directory.parent
elif current_directory.name == "anomalib":
    # This means that the notebook is run from the main anomalib directory.
    root_directory = current_directory

os.chdir(root_directory)
root_directory

PosixPath('/home/wueesmat/MT/anomalib')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
# Import the required modules
from lightning.pytorch import Trainer, seed_everything
from anomalib.data import MVTec
from anomalib.data.image.mvtec import MVTec_contaminated, make_mvtec_dataset_contaminated
from anomalib.models import Padim, Patchcore, Stfpm, Draem, EfficientAd
from anomalib.engine import Engine
from anomalib import TaskType

Could not find wandb. To use this feature, ensure that you have wandb installed.
Could not find openvino. To use this feature, ensure that you have openvino installed.
OpenVINO is not installed. Please install OpenVINO to use OpenVINOInferencer.


In [4]:
save_folder = "./results/Patchcore/"
name = "Patchcore_20240402_no_refinement.npy"
name_refined = "Patchcore_20240402_simple_refinement.npy"
file_path = os.path.join(save_folder, name)
file_path_refined = os.path.join(save_folder, name_refined)

run_arr = np.array([1]) #np.array([1, 2, 3]) #np.array([42])#np.arange(3)+1
cont_ratio_arr = np.array([0.15])#, 0.1]) #np.array([0.0, 0.05, 0.1]) #, 0.15
category_arr = np.array(["metal_nut"])#, "grid"])#np.array(["carpet", "grid", "leather", "tile", "wood", "bottle", "cable", "capsule", "hazelnut", "metal_nut", "pill", "screw", "toothbrush", "transistor", "zipper"])#np.array(["carpet", "grid"]) #
results_arr = np.empty([run_arr.shape[0], cont_ratio_arr.shape[0], category_arr.shape[0]])
results_arr_refined = np.empty([run_arr.shape[0], cont_ratio_arr.shape[0], category_arr.shape[0]])

for idx_run, run in enumerate(run_arr):
    for idx_cont_ratio, cont_ratio in enumerate(cont_ratio_arr):
        for idx_category, category in enumerate(category_arr):

            # Define seed, datamodule, model and engine
            seed_everything(run, workers=True)
            datamodule = MVTec_contaminated(category=category, cont_ratio=cont_ratio, run=run, idx=[])
            model = Patchcore() #Padim(backbone="resnet18", n_features=100, layers=["layer1", "layer2", "layer3"])
            engine = Engine(task=TaskType.CLASSIFICATION, image_metrics=["AUROC", "AUPR", "F1Score"], max_epochs=10, devices=1)

            # Train the model
            engine.fit(datamodule=datamodule, model=model)

            # load best model from checkpoint before evaluating
            test_results = engine.test(
                model=model,
                datamodule=datamodule,
                ckpt_path=engine.trainer.checkpoint_callback.best_model_path,
                verbose=False
            )
            results_arr[idx_run, idx_cont_ratio, idx_category] = test_results[0]["image_AUROC"]

            
            # Refine training set 
            prediction_dataset = datamodule.get_train_dataset()
            predictions = engine.predict(model=model, dataset=prediction_dataset) # Make predictions on training set
            prediction_scores = np.array([d["pred_scores"][0] for d in predictions]).tolist() # Get list of prediction scores
            sorted_indices = sorted(range(len(prediction_scores)), key=lambda i: prediction_scores[i]) # Sort the training samples based on prediction scores
            selected_indices = sorted_indices[:int((1-cont_ratio) * len(sorted_indices))] # Select the 90% lowest prediction score samples

            
            # Define seed, datamodule, model and engine
            seed_everything(run, workers=True)
            datamodule_refined = MVTec_contaminated(category=category, cont_ratio=cont_ratio, run=run, idx=selected_indices)
            model_refined = Patchcore()
            engine_refined = Engine(task=TaskType.CLASSIFICATION, image_metrics=["AUROC", "AUPR", "F1Score"], max_epochs=10, devices=1)

            # Train the model
            engine_refined.fit(datamodule=datamodule_refined, model=model_refined)

            # load best model from checkpoint before evaluating
            test_results_refined = engine_refined.test(
                model=model_refined,
                datamodule=datamodule_refined,
                ckpt_path=engine_refined.trainer.checkpoint_callback.best_model_path,
                verbose=False
            )
            results_arr_refined[idx_run, idx_cont_ratio, idx_category] = test_results_refined[0]["image_AUROC"]
            
            print(run)
            print(cont_ratio)
            print(category)

# Save results_arr to the specified folder
np.save(file_path, results_arr)
np.save(file_path_refined, results_arr_refined)

[rank: 0] Seed set to 1
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wueesmat/anaconda3/envs/anomalib_env/lib/pytho ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead


                                                  path  split    label  \
0    /home/wueesmat/MT/anomalib/datasets/MVTec/meta...  train     good   
1    /home/wueesmat/MT/anomalib/datasets/MVTec/meta...  train     good   
2    /home/wueesmat/MT/anomalib/datasets/MVTec/meta...  train     good   
3    /home/wueesmat/MT/anomalib/datasets/MVTec/meta...  train     good   
4    /home/wueesmat/MT/anomalib/datasets/MVTec/meta...  train     good   
..                                                 ...    ...      ...   
253  /home/wueesmat/MT/anomalib/datasets/MVTec/meta...   test     bent   
254  /home/wueesmat/MT/anomalib/datasets/MVTec/meta...   test    color   
255  /home/wueesmat/MT/anomalib/datasets/MVTec/meta...   test  scratch   
256  /home/wueesmat/MT/anomalib/datasets/MVTec/meta...   test    color   
257  /home/wueesmat/MT/anomalib/datasets/MVTec/meta...   test     flip   

                                            image_path  label_index  \
0    /home/wueesmat/MT/anomalib/datasets

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/pytorch/core/optimizer.py:180: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 24.9 M
1 | _transform            | Compose                  | 0     
2 | normalization_metrics | MinMax                   | 0     
3 | image_threshold       | F1AdaptiveThreshold      | 0     
4 | pixel_threshold       | F1AdaptiveThreshold      | 0     
5 | image_metrics         | AnomalibMetricCollection | 0     
6 | pixel_metrics         | AnomalibMetricCollection | 0     
-------------------------------------------------------------------
24.9 M    Trainable params
0         Non-trainable params
24.9 M    Total params
99.450    Total estimated model params 

Training: |                                                                                                   …

/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/pytorch/loops/optimization/automatic.py:129: `training_step` returned `None`. If this was on purpose, ignore this warning...


...

Validation: |                                                                                                 …

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
Restoring states from the checkpoint path at /home/wueesmat/MT/anomalib/results/Patchcore/MVTec_contaminated/metal_nut/v5/weights/lightning/model.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/wueesmat/MT/anomalib/results/Patchcore/MVTec_contaminated/metal_nut/v5/weights/lightning/model.ckpt


Testing: |                                                                                                    …

1
0.15
metal_nut


In [5]:
258/280

0.9214285714285714

In [8]:
test_results[0]["image_AUROC"]

0.9611570835113525

In [9]:
test_results_refined[0]["image_AUROC"]

0.9719008207321167

In [ ]:
file_path1

In [ ]:
# load best model from checkpoint before evaluating
test_results2 = engine2.test(
    model=model2,
    datamodule=datamodule2,
    ckpt_path=engine2.trainer.checkpoint_callback.best_model_path,
    verbose=False
)

In [ ]:
test_results2[0]["image_AUROC"]

In [ ]:
test_results4 = engine2.test(
    model=model2,
    datamodule=datamodule2,
    verbose=False
)

In [ ]:
test_results4[0]["image_AUROC"]

In [ ]:
results_arr1

In [ ]:
results_arr2

In [ ]:
results_arr1

In [ ]:
results_arr2

In [ ]:
%matplotlib inline 

In [ ]:
pred_scores_array = np.array([d["pred_scores"] for d in predictions])

In [ ]:
plt.get_backend()

In [ ]:
plt.hist(pred_scores_array, bins=5)  # Adjust the number of bins as needed
plt.title('Histogram of Prediction Scores')
plt.xlabel('Prediction Score')
plt.ylabel('Frequency')
plt.show()

In [ ]:
results_arr

In [ ]:
type(predictions)

In [ ]:
type(predictions[0])

In [ ]:
predictions[0]['pred_scores'][0]

In [ ]:
pred_scores_array = np.array([d["pred_scores"][0] for d in predictions])

In [ ]:
np.array([d["pred_scores"][0] for d in predictions]).tolist()

In [ ]:
# Get list of prediction scores
prediction_scores = np.array([d["pred_scores"][0] for d in predictions]).tolist()

# Sort the training samples based on prediction scores
sorted_indices = sorted(range(len(prediction_scores)), key=lambda i: prediction_scores[i])

# Select the 90% lowest prediction score samples
selected_indices = sorted_indices[:int(0.9 * len(sorted_indices))]



In [ ]:
selected_indices

In [ ]:
# Create a new dataset with the selected samples
new_train_data = [datamodule.train_data[i] for i in selected_indices]

In [ ]:
type(new_train_data)

In [ ]:
datamodule2 = datamodule
datamodule2.set_train_data(new_train_data)

In [ ]:
datamodule2

In [ ]:
seed_everything(run, workers=True)
datamodule = MVTec_contaminated(category=category, cont_ratio=cont_ratio, run=run)

datamodule2 = datamodule
datamodule2.set_train_data(new_train_data)

#model = Padim(backbone="resnet18", n_features=100, layers=["layer1", "layer2", "layer3"])
model = Patchcore()

engine = Engine(task=TaskType.CLASSIFICATION, image_metrics=["AUROC", "AUPR", "F1Score"], max_epochs=10, devices=1)

# Train the model
engine.fit(datamodule=datamodule2, model=model)




In [ ]:
new_prediction_dataset = datamodule.get_train_dataset()
predictions = engine.predict(model=model, dataset=new_prediction_dataset)

pred_scores_array = np.array([d["pred_scores"] for d in predictions])
np.percentile(pred_scores_array, q=100*(1-cont_ratio))


#######
# load best model from checkpoint before evaluating
test_results = engine.test(
    model=model,
    datamodule=datamodule,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path,
    verbose=False
)
results_arr[idx_run, idx_cont_ratio, idx_category] = test_results[0]["image_AUROC"]

In [ ]:
cont_ratio = 0.1

In [ ]:
np.percentile(pred_scores_array, q=100*(1-cont_ratio))

In [ ]:
pred_scores_array

In [ ]:
predictions

In [ ]:
pred_scores_array = np.array([d["pred_scores"] for d in predictions])

In [ ]:
[d["pred_scores"] for d in predictions]

In [ ]:
sorted_predictions = sorted(predictions, key=lambda x: x["pred_scores"])

In [ ]:
sorted_predictions[0]["pred_scores"]

In [ ]:
sorted_predictions[270]["pred_scores"]